# Methods validation: Autoregression with seasonality

Using synthetic data with known properties, we verify what behaviors the parameters of various `statsmodels.tsa` and `statsmodels.api.sm.ols` methods yield.

In [1]:
from pathlib import Path
import datetime as dt
from typing import Union, Tuple, List
from pprint import pprint
from math import ceil
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tools import eval_measures
from jinja2 import Template

In [2]:
parent_path = Path('.').resolve().parent  # Path.cwd()

## Plotting functions

In [3]:
def altair_axis_encoding(source: pd.DataFrame,
                         z_: str) -> str:
    """
    Provides Altair with the proper encoding for an axis variable in shorthand.
    Consider refactoring using the long-form alt.X('name', type='quantitative'), etc.
    
    :param source: tabular data set containing a column to be plotted
    :param z_: column to be plotted
    """  
    if source[z_].dtype in [int, float]:
        axis_encoding = z_ + ':Q'  # a continuous real-valued quantity
    elif ((pd.api.types.is_datetime64_ns_dtype(source[z_])) |
          (pd.api.types.is_period_dtype(source[z_]))
         ):
        axis_encoding = z_ + ':T'  # a time or date value
    elif source[z_].dtype == 'object':
        print(f'Column {z_} is currently of dtype object; please convert to either int, float, or datetime.')
    else:
        print(f'Column {z_} is not correctly formatted; please convert to either int, float, or datetime.')
    return axis_encoding


def altair_ts_scatter(source: pd.DataFrame,
                      x_: str,
                      y_: str,
                      tooltip_fld: str,
                      categorical_colors: bool = False,
                      _cat: str = 'darkblue',
                      x_title: str = '',
                      y_title: str = '',
                      _zero: bool = False,
                      _title: str = '',
                      h_w: Tuple[str] = (200, 400)):
    """
    Using filled circles, plots time series of transaction metrics (amount, volume, etc.) 
    on a daily resolution. To the right, plot a frequency distribution of the same variable.
    
    :param source: tabular data set containing columns to be plotted
    :param x_: column name containing datetimes or integer denoting day of week or year, week or month, etc.
    :param y_: transaction amount, volume, or other metric
    :param _cat: categorical field used to color-code symbols, or a default single color
    :param x_title: horizontal axis title
    :param y_title: vertical axis title
    :param _zero: whether to scale the vertical axis from zero (True) or on the basis of the range of values (False)
    :param _title: chart title
    :returns: altair graph (json) object 
    """   
    x_axis_encoding = altair_axis_encoding(source, x_)
    
    if not categorical_colors:
        chart = alt.Chart(source).mark_circle(opacity=0.6, color=_cat).encode(
            x=alt.X(x_axis_encoding, title=x_title, 
                    axis=alt.Axis(grid=False,
                                  ticks=True,
                                 )
                   ),
            y=alt.Y(y_ + ':Q', title=y_title, scale=alt.Scale(zero=_zero),
                    axis=alt.Axis(grid=True,
                                  ticks=True,
                                  domain=False  # axis line
                                 )
                   ),
            tooltip=altair_axis_encoding(source, tooltip_fld)
        ).properties(
            title=_title,
            height=h_w[0],
            width=h_w[1],
        )
    else:
        chart = alt.Chart(source).mark_circle(opacity=0.6).encode(
        x=alt.X(x_axis_encoding, title=x_title, 
                axis=alt.Axis(grid=False,
                              ticks=True,
                             )
               ),
        y=alt.Y(y_ + ':Q', title=y_title, scale=alt.Scale(zero=_zero),
                axis=alt.Axis(grid=True,
                              ticks=True,
                              domain=False
                             )
               ),
        color=_cat + ':N',
        tooltip=altair_axis_encoding(source, tooltip_fld)
        ).properties(
            title=_title,
            height=h_w[0],
            width=h_w[1],
        ).configure_view(strokeWidth=0)
    return chart


def altair_ts_line(source: pd.DataFrame,
                   x_: str,
                   y_: str,
                   _color: str = 'palevioletred',
                   x_title: str = '',
                   y_title: str = '',
                   _zero: bool = False,
                   _title: str = '',
                   h_w: Tuple[str] = (200, 400)):
    """
    Using symbols, plots time series of transaction metrics (amount, volume, etc.) on a daily resolution.
    To the right, plot a frequency distribution of the same variable.
    
    :param source: tabular data set containing columns to be plotted
    :param x_: column name containing datetimes or integer denoting day of week or year, week or month, etc.
    :param y_: transaction amount, volume, or other metric
    :param x_title: horizontal axis title
    :param y_title: vertical axis title
    :param _zero: whether to scale the vertical axis from zero (True) or on the basis of the range of values (False)
    :param _title: chart title
    :param cat: categorical field used to color-code symbols.
    :returns: altair graph (json) object 
    """
    x_axis_encoding = altair_axis_encoding(source, x_)
    ts_line = alt.Chart(source).mark_line(color=_color).encode(
        x=alt.X(x_axis_encoding, title=x_title,
                axis=alt.Axis(grid=False,
                              ticks=True,
                             )
               ),
        y=alt.Y(y_ + ':Q', title=y_title, scale=alt.Scale(zero=_zero),
               axis=alt.Axis(grid=True,
                              ticks=True,
                              domain=False  # axis line
                             )
               ),
    ).properties(
        title=_title,
        height=h_w[0],
        width=h_w[1],
    ) 
    return ts_line

## Create artificial data with known patterns

Create one year of daily timestamps and initialize the observations with random numbers $\in (0, 1)$.

In [4]:
def artifice(day_one: str = '2022-01-01',
             day_after_end: str = '2023-01-01',
            ) -> pd.DataFrame:
    """Generate an artificial time series of random numbers [0, 1] from
    [day_one, day_after_end).
    """
    days = np.arange(day_one, day_after_end, dtype='datetime64[D]')
    print(f'There are {len(days)} days in the period starting {days.min()} and ending {days.max()}.')
    rng = np.random.RandomState(1216)
    return pd.DataFrame({'t': days,
                         'y': rng.random(len(days))})


def week_of_month(stamp: dt.date):
    """
    Determines which week of the month a date occurs.
    
    Parameters
    ----------
    stamp : datetime stamp
    """
    # Replace the day portion of the date with 1
    first_day = stamp.replace(day=1)
    
    day_of_month = stamp.day
    adjusted_dom = day_of_month + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))


def seq_ordinal(data: pd.DataFrame,
                date_col: 'str' = 't',
               ):
    """
    """
    data = data.copy()
    data.loc[:, 'dow'] = data[date_col].dt.weekday
    data.loc[:, 'wom'] = data[date_col].apply(week_of_month)
    return data


def signal_boost(data,
                 level1_to_boost: int = 4,  # Friday
                 # level2_to_boost: Tuple = (2, 4),  # last __ of the month
                 boost: float = 2.,
                 every_other_week: bool = False,
                ):
    """
    """
    data = data.copy()
    if every_other_week:
        data.y = data.y + np.where((data.dow==level1_to_boost) &
                                   (data.wom % 2 == 0), boost, 0)
    else:
        data.y = data.y + np.where(data.dow==level1_to_boost, boost, 0)
    return data

In [5]:
ts = artifice()
print(ts[0:4])

There are 365 days in the period starting 2022-01-01 and ending 2022-12-31.
           t         y
0 2022-01-01  0.995598
1 2022-01-02  0.623411
2 2022-01-03  0.876094
3 2022-01-04  0.889378


In [6]:
def run_autoreg(data: pd.DataFrame,
                y: str,
                lags_: int,
                seasonal_: bool,
                period_: int = 0,
                plot_fit: bool = True,
                exhibit_no: str = '',
                ):
    data = data.copy()
    auto_reg = AutoReg(data[y],
                       lags=lags_,
                       trend='t',
                       seasonal=seasonal_,
                       period=period_
                       )
    auto_reg0 = auto_reg.fit()
    model_params_md = auto_reg0.params.to_markdown()
    data.loc[:, 'y_hat'] = auto_reg0.predict()
    maedf = data.dropna()
    mae = eval_measures.meanabs(maedf[y], maedf.y_hat)
    
    if plot_fit:
        pan_zoom = alt.selection_interval(bind='scales')
        c0 = altair_ts_scatter(data, 't', 'y', 't')
        c2 = altair_ts_line(data, 't', 'y_hat', 't',
                            _title=f"{exhibit_no}{lags_} lags, period {period_}: MAE {mae:.3f}")
        return auto_reg0, model_params_md, data, (c0 + c2).add_selection(pan_zoom)
    else:
        return auto_reg0, model_params_md, data

### Optional: Plot raw data

In [7]:
stage0 = altair_ts_scatter(ts, 't', 'y', 't', _title='Figure 0. Random scatter.')

In [8]:
stage0

alt.Chart(...)

### Encode a time characteristic, such as day of the week (dow) and boost the signal on certain days (or weeks, etc.).

In [9]:
ts2 = seq_ordinal(ts)
ts2 = signal_boost(ts2,every_other_week=False)
s1data = ts2[0:19].to_markdown(index=False)
# print(ts[0:29])

In [10]:
ts.head(10)

,t,y
0,2022-01-01,0.995598
1,2022-01-02,0.623411
2,2022-01-03,0.876094
3,2022-01-04,0.889378
4,2022-01-05,0.386762
5,2022-01-06,0.689920
6,2022-01-07,0.992853
7,2022-01-08,0.468042
8,2022-01-09,0.945584
9,2022-01-10,0.638123


In [11]:
ts2.head(10)

,t,y,dow,wom
0,2022-01-01,0.995598,5,1
1,2022-01-02,0.623411,6,1
2,2022-01-03,0.876094,0,2
3,2022-01-04,0.889378,1,2
4,2022-01-05,0.386762,2,2
5,2022-01-06,0.689920,3,2
6,2022-01-07,2.992853,4,2
7,2022-01-08,0.468042,5,2
8,2022-01-09,0.945584,6,2
9,2022-01-10,0.638123,0,3


### Option to remove some points

In [12]:
def abscond(data,
            points: int = 10,
           ):
    
    rng = np.random.RandomState(1216)
    idx_mask = rng.randint(0, len(data), points)
    data = data[~data.index.isin(idx_mask)]
    print(f'{points} points randomly removed at positions {idx_mask}, leaving {len(data)} observations.')
    
    # Inspect that a row was removed
    print(data[idx_mask[0] - 5 : idx_mask[0] + 2])
    return data, idx_mask

## Fit autoregressive model

With `seasonal=False`, we obtain an upwardly trending oscillation.

In [13]:
s1model, s1params, _, stage1 = run_autoreg(ts2, 'y',
                                           lags_=2,
                                           seasonal_=False,
                                           exhibit_no='Figure 1. '
                                          )
print(s1model.params)
stage1

trend    0.002558
y.L1     0.105590
y.L2     0.100251
dtype: float64


alt.LayerChart(...)

Enable `seasonal`, and we get the expected level baseline with weekly peaks:

In [14]:
_, s2params, _, stage2 = run_autoreg(ts2, 'y',
                                     lags_=4,
                                     seasonal_=True,
                                     period_=7,
                                     exhibit_no='Figure 2. '
                                    )
stage2

alt.LayerChart(...)

If we increase `lags=7` to include the weekly effect, we get almost as good a model as with seasonal terms:

In [15]:
_, s3params, _, stage3 = run_autoreg(ts2, 'y',
                                     lags_=7,
                                     seasonal_=False,
                                     exhibit_no='Figure 3. '
                                    )
stage3

alt.LayerChart(...)

Look at the coefficients by applying the `.params` method to the fitted model object; e.g., with `lags=2`, `seasonal=True`, and `period=7`: 

In [16]:
s4model, s4params, _, stage4 = run_autoreg(ts2, 'y',
                                           lags_=2,
                                           seasonal_=True,
                                           period_=7,
                                           exhibit_no='Figure 4. '
                                          )
print(s4model.params)

trend     0.000016
s(1,7)    0.429604
s(2,7)    0.391794
s(3,7)    0.425179
s(4,7)    0.459669
s(5,7)    0.424455
s(6,7)    0.467097
s(7,7)    2.389252
y.L1      0.028575
y.L2      0.053987
dtype: float64


If we remove a small number of points at random such that there are gaps in the index, the model falls apart (not shown).  Can we make the seasonal regression algorithm aware that observations are made on calendar days?

In [17]:
ts3, idx_mask = abscond(ts2, points=5)

5 points randomly removed at positions [288 284 344  99 361], leaving 360 observations.
             t         y  dow  wom
285 2022-10-13  0.198405    3    3
286 2022-10-14  2.460687    4    3
287 2022-10-15  0.633509    5    3
289 2022-10-17  0.016791    0    4
290 2022-10-18  0.154824    1    4
291 2022-10-19  0.322947    2    4
292 2022-10-20  0.097235    3    4


Before and after applying `.asfreq('d')`:

In [18]:
s5_before_asfreq = ts3[280:292].to_markdown(index=False)
ts3[280:292]

,t,y,dow,wom
281,2022-10-09,0.722428,6,2
282,2022-10-10,0.291553,0,3
283,2022-10-11,0.369910,1,3
285,2022-10-13,0.198405,3,3
286,2022-10-14,2.460687,4,3
287,2022-10-15,0.633509,5,3
289,2022-10-17,0.016791,0,4
290,2022-10-18,0.154824,1,4
291,2022-10-19,0.322947,2,4
292,2022-10-20,0.097235,3,4


In [19]:
ts3 = ts3.set_index('t').asfreq('d')

In [20]:
s5_after_asfreq = ts3[280:292].to_markdown(index=False)
ts3[280:292]

,y,dow,wom
t,,,
2022-10-08,0.721330,5.0,2.0
2022-10-09,0.722428,6.0,2.0
2022-10-10,0.291553,0.0,3.0
2022-10-11,0.369910,1.0,3.0
2022-10-12,NaN,NaN,NaN
2022-10-13,0.198405,3.0,3.0
2022-10-14,2.460687,4.0,3.0
2022-10-15,0.633509,5.0,3.0
2022-10-16,NaN,NaN,NaN


try a `PeriodIndex`, a subclass of `Index` that is regularly spaced:  We still have gaps in the time series, but no `nan` have been inserted:

In [21]:
ts4 = artifice()
ts4 = seq_ordinal(ts4)
ts4 = signal_boost(ts4, every_other_week=False)
ts4, idx_mask = abscond(ts4, points=5)
ts4.set_index('t', inplace=True)

There are 365 days in the period starting 2022-01-01 and ending 2022-12-31.
5 points randomly removed at positions [288 284 344  99 361], leaving 360 observations.
             t         y  dow  wom
285 2022-10-13  0.198405    3    3
286 2022-10-14  2.460687    4    3
287 2022-10-15  0.633509    5    3
289 2022-10-17  0.016791    0    4
290 2022-10-18  0.154824    1    4
291 2022-10-19  0.322947    2    4
292 2022-10-20  0.097235    3    4


In [22]:
ts4.index

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
               '2022-01-09', '2022-01-10',
               ...
               '2022-12-21', '2022-12-22', '2022-12-23', '2022-12-24',
               '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', name='t', length=360, freq=None)

In [23]:
ts4.index = pd.DatetimeIndex(ts4.index).to_period('D')

In [24]:
ts4.index

PeriodIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
             '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
             '2022-01-09', '2022-01-10',
             ...
             '2022-12-21', '2022-12-22', '2022-12-23', '2022-12-24',
             '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-29',
             '2022-12-30', '2022-12-31'],
            dtype='period[D]', name='t', length=360)

In [25]:
s5_periodIndex = ts4[280:292].to_markdown(index=False)  # or part of sequence with gaps
ts4[280:292]

,y,dow,wom
t,,,
2022-10-09,0.722428,6,2
2022-10-10,0.291553,0,3
2022-10-11,0.369910,1,3
2022-10-13,0.198405,3,3
2022-10-14,2.460687,4,3
2022-10-15,0.633509,5,3
2022-10-17,0.016791,0,4
2022-10-18,0.154824,1,4
2022-10-19,0.322947,2,4


It is unnecessary to include `missing='drop'` in `AutoReg()`, since the data has no missing values (it is probably good practice to include `missing='raise'` as a check).  `.fit()` runs and `.predict()` returns values without alterring the input data structure: it still has a `PeriodIndex`:

In [26]:
_, s5params, ts4_output, stage5 = run_autoreg(ts4.reset_index(), 'y',
                                       lags_=7,
                                       seasonal_=False,
                                      )

In [27]:
ts4_outputmd = ts4_output[0:9].to_markdown(index=False)
ts4_output.head(10)

,t,y,dow,wom,y_hat
0,2022-01-01,0.995598,5,1,NaN
1,2022-01-02,0.623411,6,1,NaN
2,2022-01-03,0.876094,0,2,NaN
3,2022-01-04,0.889378,1,2,NaN
4,2022-01-05,0.386762,2,2,NaN
5,2022-01-06,0.689920,3,2,NaN
6,2022-01-07,2.992853,4,2,NaN
7,2022-01-08,0.468042,5,2,0.686514
8,2022-01-09,0.945584,6,2,0.655461
9,2022-01-10,0.638123,0,3,0.759031


In [28]:
ts4.index = ts4.index.to_timestamp()

In [29]:
_, _, _, stage5 = run_autoreg(ts4.reset_index(), 'y',
                                       lags_=7,
                                       seasonal_=False,
                                       exhibit_no='Figure 5. '
                                      )

In [30]:
stage5

alt.LayerChart(...)

Using `.asfreq` to conform a daily time series of 365 points minus 10 removed at
random to a `DatetimeIndex`-ed dataframe with `freq='D'`, we get a good fit.

In [31]:
ts5 = artifice()
ts5 = seq_ordinal(ts5)
ts5 = signal_boost(ts5, every_other_week=False)
ts5, _ = abscond(ts5)
ts5 = ts5.set_index('t').asfreq('d')
ts5.fillna(0, inplace=True)
_, s6params, _, stage6 = run_autoreg(ts5.reset_index(), 'y',
                                     lags_=2,
                                     seasonal_=True,
                                     period_=7,
                                     exhibit_no='Figure 6. '
                                    )

There are 365 days in the period starting 2022-01-01 and ending 2022-12-31.
10 points randomly removed at positions [288 284 344  99 361 126 284  58 319 312], leaving 356 observations.
             t         y  dow  wom
287 2022-10-15  0.633509    5    3
289 2022-10-17  0.016791    0    4
290 2022-10-18  0.154824    1    4
291 2022-10-19  0.322947    2    4
292 2022-10-20  0.097235    3    4
293 2022-10-21  2.161951    4    4
294 2022-10-22  0.930828    5    4


In [32]:
stage6

alt.LayerChart(...)

What if the signal boost occurred _every other_ Friday?

In [33]:
ts6 = artifice()
ts6 = seq_ordinal(ts6)
ts6 = signal_boost(ts6, every_other_week=False)
ts6 = signal_boost(ts6,
                  level1_to_boost=4,  # Friday
                  boost=2.,
                  every_other_week=True,
                 )
_, s7params, _, stage7 = run_autoreg(ts6, 'y',
                                     lags_=2,
                                     seasonal_=True,
                                     period_=7,
                                     exhibit_no='Figure 7. '
                                    )

There are 365 days in the period starting 2022-01-01 and ending 2022-12-31.


In [34]:
compound = (stage0 | stage1) & (stage2 | stage3) & (stage5 | stage7)

In [35]:
compound

alt.VConcatChart(...)

# Put content into template

In [42]:
p = parent_path/'assets/practical_ts01t.md'
template = Template(p.read_text())
o = parent_path/'content/posts/practical_ts01.md'
o.write_text(template.render({'alt': alt,
                              'stage0': 'Figure 0 (appended). A random sequence.', # stage0.to_html(),
                              's1data': s1data,
                              'stage1': 'Figure 1 (appended). A random sequence, where the observation on every Friday is artificially increased by the same amount, fitted with a two-lag autoregressive model.', # stage1.to_html(),
                              'stage2': 'Figure 2 (appended).', # stage2.to_html(),
                              'stage3': 'Figure 3 (appended).', # stage3.to_html(),
                              's4params': s4params,
                              's5_before_asfreq': s5_before_asfreq,
                              's5_after_asfreq': s5_after_asfreq,
                              's5_periodIndex': s5_periodIndex,
                              'ts5md': ts4_outputmd,
                              'stage5': 'Figure 5 (appended).', # stage5.to_html(),
                              's5params': s5params,
                              's6params': s6params,
                              'stage7': compound.to_html(),
                             }
                            )
            )

240480

Troubleshoot jinja, hugo issues

In [ ]:
# parent_path

In [41]:
p = parent_path/'assets/aa1chart_t.md'
template = Template(p.read_text())
o = parent_path/'content/posts/aa1chart.md'
o.write_text(template.render({
                              'stage0': stage0.to_html(),
                              'stage7': stage7.to_html(),
                             }
                            )
            )

62516

In [ ]:
# stage0

In [ ]:
# Path.cwd()